In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import numpy as np
import os
import lzma
import pickle
import polars as pl

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score

In [2]:
pl.Config.set_tbl_rows(-1)
pl.Config.set_tbl_cols(-1)
pl.Config.set_tbl_width_chars(-1)

polars.config.Config

In [3]:
class CarPilotNet(nn.Module):
    def __init__(self, n_inputs=16, n_outputs=4):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_inputs, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, n_outputs)  # raw logits for [fwd, back, left, right]
        )
    def forward(self, x):
        return self.net(x)

In [4]:
def extract_data(record_dir = "./records/"):
    all_records = []  # Accumulate all records here

    for filename in os.listdir(record_dir):
        if not filename.endswith(".npz"):
            continue

        input_path = os.path.join(record_dir, filename)
        try:
            with lzma.open(input_path, "rb") as _f:
                snapshots = pickle.load(_f)
        except Exception as e:
            print(f"❌ Failed to load {filename}: {e}")
            continue

        for _idx, s in enumerate(snapshots):
            record = {
                "record": filename,
                "frame_idx": _idx,
                "forward": s.current_controls[0],
                "back": s.current_controls[1],
                "left": s.current_controls[2],
                "right": s.current_controls[3],
                "car_speed": s.car_speed,
                **{f"raycast_{i}": float(d) for i, d in enumerate(s.raycast_distances)},
            }
            all_records.append(record)

    # Create a single Polars DataFrame from all records
    try:
        df = pl.DataFrame(all_records).sort("record")
        print(f"✅ Successfully created combined Polars DataFrame with {len(df)} rows")
    except Exception as e:
        print(f"❌ Error creating combined DataFrame: {e}")
    return df.filter((pl.col("raycast_7") >= 50) & (pl.col("forward") == 0)
)


In [9]:
# ---------------------------------
# Load or simulate your dataset
# ---------------------------------
n_rays = 15
n_samples = 5000

# Instantiate
model = CarPilotNet(n_inputs=n_rays+1, n_outputs=4)

# Extract data

df = extract_data()
X = df.select(pl.exclude(['forward', 'back', 'left', 'right', 'record', 'frame_idx']))

# build expressions to replace car_speed and raycast_* columns
raycast_exprs = [pl.col(f"raycast_{i}") / 100 for i in range(n_rays)]
car_speed_expr = (pl.col("car_speed") + 50) / 100

# apply the transformations (expressions replace the existing columns)
X = X.with_columns([car_speed_expr] + raycast_exprs).to_numpy()
    
Y = df.select(pl.col(['forward','back','left','right'])).to_numpy()
#[fwd, back, left, right]


# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32)
dataset = TensorDataset(X_tensor, Y_tensor)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

# ---------------------------------
# Initialize model, loss, optimizer
# ---------------------------------
model = CarPilotNet(n_inputs=X.shape[1])
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# ---------------------------------
# Training loop
# ---------------------------------
epochs = 100
# assume: logits shape (B,4), yb shape (B,4), xb shape (B, n_inputs)
# xb column order: [car_speed_normalized, raycast_0..raycast_14]
# car_speed_original = xb[:,0]*100 - 50  # if you stored normalized speed as (speed+50)/100
# hyperparameters
penalty_weight_pos = 1.2   # your existing positive penalty (encourage forward when slow)
penalty_weight_neg = 1.0   # new negative penalty (discourage forward when too fast)
speed_low = -5.0
speed_high = 5.0

bce_fn = nn.BCEWithLogitsLoss()

for epoch in range(epochs):
    total_loss = 0.0
    for xb, yb in loader:
        logits = model(xb)
        base_loss = criterion(logits, yb)

        # recover original speed from normalized column 0 (if you used (speed+50)/100)
        speed_orig = xb[:, 0] * 100.0 - 50.0

        # POSITIVE penalty: encourage forward=1 when speed in [speed_low, speed_high] and GT forward == 0
        mask_pos = (speed_orig >= speed_low) & (speed_orig <= speed_high) & (yb[:, 0] == 0)

        extra_loss = 0.0

        if mask_pos.any():
            logits_fwd_pos = logits[mask_pos, 0].unsqueeze(1)    # shape (M,1)
            target_ones = torch.ones_like(logits_fwd_pos)
            extra_loss_pos = bce_fn(logits_fwd_pos, target_ones)
            extra_loss = extra_loss + penalty_weight_pos * extra_loss_pos

        # NEGATIVE penalty: discourage forward=1 when speed is too high and GT forward == 1
        mask_neg = (speed_orig > speed_high) & (yb[:, 0] == 1)

        if mask_neg.any():
            logits_fwd_neg = logits[mask_neg, 0].unsqueeze(1)    # shape (N,1)
            target_zeros = torch.zeros_like(logits_fwd_neg)
            extra_loss_neg = bce_fn(logits_fwd_neg, target_zeros)
            extra_loss = extra_loss + penalty_weight_neg * extra_loss_neg

        # final loss
        if isinstance(extra_loss, float) and extra_loss == 0.0:
            loss = base_loss
        else:
            loss = base_loss + extra_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:03d} | Loss: {total_loss/len(loader):.4f}")


✅ Successfully created combined Polars DataFrame with 23058 rows
Epoch 010 | Loss: 0.2462
Epoch 020 | Loss: 0.2175
Epoch 030 | Loss: 0.2034
Epoch 040 | Loss: 0.2170
Epoch 050 | Loss: 0.1955
Epoch 060 | Loss: 0.1750
Epoch 070 | Loss: 0.1694
Epoch 080 | Loss: 0.1760
Epoch 090 | Loss: 0.1644
Epoch 100 | Loss: 0.1899


In [10]:
torch.save(model.state_dict(), "./model/model.pth")

In [7]:
rays = df.select([f"raycast_{i}" for i in range(n_rays)])
speed = df.select('car_speed')
print(speed.max())
print(speed.min())

shape: (1, 1)
┌───────────┐
│ car_speed │
│ ---       │
│ f64       │
╞═══════════╡
│ 49.879276 │
└───────────┘
shape: (1, 1)
┌───────────┐
│ car_speed │
│ ---       │
│ f64       │
╞═══════════╡
│ 0.0       │
└───────────┘


In [ ]:
import os
import lzma
import pickle
import numpy as np
from types import SimpleNamespace
from typing import Optional, Union

def sample_rays(rng: np.random.Generator):
    center = rng.integers(0, 3)
    base = [100.0]*15
    if center == 0:
        for i in range(15):
            base[i] = max(20.0, min(100.0, rng.normal(85, 8)))
    elif center == 1:
        for i in range(15):
            base[i] = max(5.0, min(100.0, rng.normal(60, 20)))
    else:
        for i in range(15):
            angle = abs(i-7)/7.0
            base[i] = max(1.0, min(100.0, rng.normal(20 + 80*angle, 12)))
    for _ in range(rng.integers(0, 3)):
        idx = rng.integers(0, 15)
        drop = rng.uniform(5,60)
        base[idx] = max(0.5, base[idx] - drop)
    return [float(round(x,3)) for x in base]

def label_from_state(rays, speed,
                     rng: Optional[np.random.Generator] = None,
                     flip_prob: float = 0.02,
                     front_threshold: float = 5.0,
                     side_diff_threshold: float = 6.0):
    """
    Deterministic when `rng` is a numpy Generator created with a fixed seed.
    Parameters:
    - rays: sequence of distance readings (length >= 15 assumed)
    - speed: scalar speed
    - rng: optional np.random.Generator for deterministic randomness. If None, function is deterministic with no flips.
    - flip_prob: probability to randomly flip each control bit (applied independently)
    - front_threshold: distance threshold to trigger 'back' (=1)
    - side_diff_threshold: difference between left/right means to steer
    Returns: [f, b, l, r] as ints
    """
    # base deterministic decisions
    f = b = l = r = 0
    front = min(rays[6:9])
    left_mean = sum(rays[0:5]) / 5.0
    right_mean = sum(rays[10:15]) / 5.0

    if front < front_threshold:
        b = 1
    else:
        if speed < 5:
            f = 1
    if speed > 10:
        f = 0

    if left_mean < right_mean - side_diff_threshold:
        r = 1
    elif right_mean < left_mean - side_diff_threshold:
        l = 1

    # apply small random flips only if rng provided
    if rng is not None and flip_prob > 0.0:
        if rng.random() < flip_prob:
            l = 1 - l
        if rng.random() < flip_prob:
            r = 1 - r
        if rng.random() < flip_prob:
            f = 1 - f
        if rng.random() < flip_prob:
            b = 1 - b

    return [int(f), int(b), int(l), int(r)]

def make_synthetic_snapshots(n_snapshots, seed: Optional[Union[int, np.integer]] = None, flip_prob: float = 0.02):
    """
    Creates snapshots reproducibly when `seed` is provided.
    - seed: integer or None. If integer, uses np.random.default_rng(seed) for all sampling.
    - flip_prob: forwarded to label_from_state for controlled randomness.
    """
    rng = np.random.default_rng(seed)  # modern, isolated RNG best practice.
    snapshots = []
    for _ in range(n_snapshots):
        rays = sample_rays(rng)
        speed = float(round(rng.uniform(-10, 50), 3))
        controls = label_from_state(rays, speed, rng=rng, flip_prob=flip_prob)
        s = SimpleNamespace(
            current_controls=controls,
            car_speed=speed,
            raycast_distances=list(rays)
        )
        snapshots.append(s)
    return snapshots

def save_snapshots_lzma_pickle(snapshots, out_path):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with lzma.open(out_path, "wb") as f:
        pickle.dump(snapshots, f)

    # writes the same format your extract_data expects
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with lzma.open(out_path, "wb") as f:
        pickle.dump(snapshots, f)

# Example usage: create 1 synthetic record file with 100 frames
snapshots = make_synthetic_snapshots(20000, seed=42)
save_snapshots_lzma_pickle(snapshots, "./records/synth_record_0001.npz")
